# What the project is about
The goal of this project: 

Help our developers understand what type of apps are likely to attract more users on Google Play and the App Store. To do this, we'll need to collect and analyze data about mobile apps available on Google Play and the App Store.

In [1]:
import pandas as pd
from csv import reader

In [2]:
### The Google Play data set ###
android=pd.read_csv('googleplaystore.csv')
### The App Store data set ###
ios=pd.read_csv('AppleStore.csv')

## Deleting Wrong Data

The Google Play data set has a dedicated discussion section, and we can see that one of the discussions outlines an error for row 10472. Let's print this row and compare it against the header and another row that is correct.

In [3]:
android.iloc[[10472]]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


The row 10472 corresponds to the app Life Made WI-Fi Touchscreen Photo Frame, and we can see that the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5 (as mentioned in the discussions section, this problem is caused by a missing value in the 'Category' column). As a consequence, we'll delete this row.

In [4]:
## we can check whether we have another outlier by using query 
android.query("Rating>5")

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


So we need to delete this outlier

In [5]:
android.drop(index=10472, axis=0,inplace=True)

In [6]:
android.reset_index(inplace=True,drop=True)

## Removing Duplicate Entries
<h2> Part One </h2> <br>
If we explore the Google Play data set long enough, we'll find that some apps have more than one entry. For instance, the application Instagram has four entries:

In [7]:
android.query("App=='Instagram'")

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
2545,Instagram,SOCIAL,4.5,66577313,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"July 31, 2018",Varies with device,Varies with device
2604,Instagram,SOCIAL,4.5,66577446,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"July 31, 2018",Varies with device,Varies with device
2611,Instagram,SOCIAL,4.5,66577313,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"July 31, 2018",Varies with device,Varies with device
3909,Instagram,SOCIAL,4.5,66509917,Varies with device,"1,000,000,000+",Free,0,Teen,Social,"July 31, 2018",Varies with device,Varies with device


## Lets count duplicated App value

In [8]:
app_list=list(android["App"])
duplicate_apps = []
unique_apps = []
for app in app_list:
    
    if app in unique_apps:
        duplicate_apps.append(app)
    else:
        unique_apps.append(app)
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:15])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


## We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. One thing we could do is remove the duplicate rows randomly, but we could probably find a better way.

If you examine the rows we printed two cells above for the Instagram app, the main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show that the data was collected at different times. We can use this to build a criterion for keeping rows. We won't remove rows randomly, but rather we'll keep the rows that have the highest number of reviews because the higher the number of reviews, the more reliable the ratings.

To do that, we will:

Create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app
Use the dictionary to create a new data set, which will have only one entry per app (and we only select the apps with the highest number of reviews)

In [9]:
reviews_max = {}
android_values_list=android.values.tolist()

for app in android_values_list:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

In [10]:
reviews_max[name]
name

'iHoroscope - 2018 Daily Horoscope & Astrology'

## Now, let's use the reviews_max dictionary to remove the duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews. In the code cell below:

We start by initializing two empty lists, android_clean and already_added. <br>
We loop through the android data set, and for every iteration: <br>
We isolate the name of the app and the number of reviews. <br>
We add the current row (app) to the android_clean list, and the app name (name) to the already_added list if: <br>
The number of reviews of the current app matches the number of reviews of that app as described in the reviews_max dictionary; and <br>
The name of the app is not already in the already_added list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for reviews_max[name] == n_reviews, we'll still end up with duplicate entries for some apps.

In [11]:
android_clean = []
already_added = []

for app in android_values_list:
    name = app[0]
    n_reviews = float(app[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name) # make sure this is inside the if block

In [12]:
app=android_values_list
app[0][0]

'Photo Editor & Candy Camera & Grid & ScrapBook'

<h1>Removing Non-English Apps<h\1>
<h2>Part One</h2>
If you explore the data sets enough, you'll notice the names of some of the apps suggest they are not directed toward an English-speaking audience. Below, we see a couple of examples from both data sets:

In [13]:
ios_list=ios.values.tolist()
ios_list

[[284882215,
  'Facebook',
  389879808,
  'USD',
  0.0,
  2974676,
  212,
  3.5,
  3.5,
  '95.0',
  '4+',
  'Social Networking',
  37,
  1,
  29,
  1],
 [389801252,
  'Instagram',
  113954816,
  'USD',
  0.0,
  2161558,
  1289,
  4.5,
  4.0,
  '10.23',
  '12+',
  'Photo & Video',
  37,
  0,
  29,
  1],
 [529479190,
  'Clash of Clans',
  116476928,
  'USD',
  0.0,
  2130805,
  579,
  4.5,
  4.5,
  '9.24.12',
  '9+',
  'Games',
  38,
  5,
  18,
  1],
 [420009108,
  'Temple Run',
  65921024,
  'USD',
  0.0,
  1724546,
  3842,
  4.5,
  4.0,
  '1.6.2',
  '9+',
  'Games',
  40,
  5,
  1,
  1],
 [284035177,
  'Pandora - Music & Radio',
  130242560,
  'USD',
  0.0,
  1126879,
  3594,
  4.0,
  4.5,
  '8.4.1',
  '12+',
  'Music',
  37,
  4,
  1,
  1],
 [429047995,
  'Pinterest',
  74778624,
  'USD',
  0.0,
  1061624,
  1814,
  4.5,
  4.0,
  '6.26',
  '12+',
  'Social Networking',
  37,
  5,
  27,
  1],
 [282935706,
  'Bible',
  92774400,
  'USD',
  0.0,
  985920,
  5320,
  4.5,
  5.0,
  '7.5.1',

In [14]:
print(ios_list[813][1])
print(ios_list[6731][1])

print(android_clean[4412][0])
print(android_clean[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜
中国語 AQリスニング
لعبة تقدر تربح DZ


We're not interested in keeping these kind of apps, so we'll remove them. One way to go about this is to remove each app whose name contains a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.).

All these characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that to build a function that checks an app name and tells us whether it contains non-ASCII characters.

We built this function below, and we use the built-in ord() function to find out the corresponding encoding number of each character.

In [15]:
def is_english(string):
    
    for character in string:
        if ord(character) > 127:
            return False
    
    return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


The function seems to work fine, but some English app names use emojis or other symbols (™, — (em dash), – (en dash), etc.) that fall outside of the ASCII range. Because of this, we'll remove useful apps if we use the function in its current form.

In [16]:
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

print(ord('™'))
print(ord('😜'))

False
False
8482
128540


<h2>Part Two
</h2>
To minimize the impact of data loss, we'll only remove an app if its name has more than three non-ASCII characters:



In [17]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(str(character)) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
True


The function is still not perfect, and very few non-English apps might get past our filter, but this seems good enough at this point in our analysis — we shouldn't spend too much time on optimization at this point.

Below, we use the is_english() function to filter out the non-English apps for both data sets:

In [18]:
android_english=[]
ios_english=[]

for app in android_clean:
    name=app[0]
    if is_english(name):
        android_english.append(app)

for app in ios_list:
    name=app[1]
    if is_english(name):
        ios_english.append(app)

android_english

[['Photo Editor & Candy Camera & Grid & ScrapBook',
  'ART_AND_DESIGN',
  4.1,
  '159',
  '19M',
  '10,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design',
  'January 7, 2018',
  '1.0.0',
  '4.0.3 and up'],
 ['U Launcher Lite – FREE Live Cool Themes, Hide Apps',
  'ART_AND_DESIGN',
  4.7,
  '87510',
  '8.7M',
  '5,000,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design',
  'August 1, 2018',
  '1.2.4',
  '4.0.3 and up'],
 ['Sketch - Draw & Paint',
  'ART_AND_DESIGN',
  4.5,
  '215644',
  '25M',
  '50,000,000+',
  'Free',
  '0',
  'Teen',
  'Art & Design',
  'June 8, 2018',
  'Varies with device',
  '4.2 and up'],
 ['Pixel Draw - Number Art Coloring Book',
  'ART_AND_DESIGN',
  4.3,
  '967',
  '2.8M',
  '100,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design;Creativity',
  'June 20, 2018',
  '1.1',
  '4.4 and up'],
 ['Paper flowers instructions',
  'ART_AND_DESIGN',
  4.4,
  '167',
  '5.6M',
  '50,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design',
  'March 26, 2017',
  '1.0'

In [19]:
android_english

[['Photo Editor & Candy Camera & Grid & ScrapBook',
  'ART_AND_DESIGN',
  4.1,
  '159',
  '19M',
  '10,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design',
  'January 7, 2018',
  '1.0.0',
  '4.0.3 and up'],
 ['U Launcher Lite – FREE Live Cool Themes, Hide Apps',
  'ART_AND_DESIGN',
  4.7,
  '87510',
  '8.7M',
  '5,000,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design',
  'August 1, 2018',
  '1.2.4',
  '4.0.3 and up'],
 ['Sketch - Draw & Paint',
  'ART_AND_DESIGN',
  4.5,
  '215644',
  '25M',
  '50,000,000+',
  'Free',
  '0',
  'Teen',
  'Art & Design',
  'June 8, 2018',
  'Varies with device',
  '4.2 and up'],
 ['Pixel Draw - Number Art Coloring Book',
  'ART_AND_DESIGN',
  4.3,
  '967',
  '2.8M',
  '100,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design;Creativity',
  'June 20, 2018',
  '1.1',
  '4.4 and up'],
 ['Paper flowers instructions',
  'ART_AND_DESIGN',
  4.4,
  '167',
  '5.6M',
  '50,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design',
  'March 26, 2017',
  '1.0'

## Isolating the Free Apps

As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. 
Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. 
Below, we isolate the free apps for both our data sets.

In [20]:
android_final = []
ios_final = []

for app in android_clean:
    price=app[7]
    if app[7]=='0':
        android_final.append(app)
for app in ios_list:
    price=app[4]
    if price==0:
        ios_final.append(app)

In [21]:
ios_final

[[284882215,
  'Facebook',
  389879808,
  'USD',
  0.0,
  2974676,
  212,
  3.5,
  3.5,
  '95.0',
  '4+',
  'Social Networking',
  37,
  1,
  29,
  1],
 [389801252,
  'Instagram',
  113954816,
  'USD',
  0.0,
  2161558,
  1289,
  4.5,
  4.0,
  '10.23',
  '12+',
  'Photo & Video',
  37,
  0,
  29,
  1],
 [529479190,
  'Clash of Clans',
  116476928,
  'USD',
  0.0,
  2130805,
  579,
  4.5,
  4.5,
  '9.24.12',
  '9+',
  'Games',
  38,
  5,
  18,
  1],
 [420009108,
  'Temple Run',
  65921024,
  'USD',
  0.0,
  1724546,
  3842,
  4.5,
  4.0,
  '1.6.2',
  '9+',
  'Games',
  40,
  5,
  1,
  1],
 [284035177,
  'Pandora - Music & Radio',
  130242560,
  'USD',
  0.0,
  1126879,
  3594,
  4.0,
  4.5,
  '8.4.1',
  '12+',
  'Music',
  37,
  4,
  1,
  1],
 [429047995,
  'Pinterest',
  74778624,
  'USD',
  0.0,
  1061624,
  1814,
  4.5,
  4.0,
  '6.26',
  '12+',
  'Social Networking',
  37,
  5,
  27,
  1],
 [282935706,
  'Bible',
  92774400,
  'USD',
  0.0,
  985920,
  5320,
  4.5,
  5.0,
  '7.5.1',

## Most Common Apps by Genre

Part One
As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three ste
s:

Build a minimal Android version of the app, and add it to Google <br> Play.
If the app has a good response from users, we then develop it f <br>urther.
If the app is profitable after six months, we also build an iOS version of the app and add it to the Ap <br>p Store.
Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both mar<br>kets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gami <br>fication.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

Part Two
We'll build two functions we can use to analyze the frequency tables:

One function to generate frequency tables that  <br>show percentages
Another function that we can use to display the percentages in a descending order descending order

In [51]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for row in table_sorted:
        print(row[1],":",row[0])

We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).



In [68]:
display_table(android_final, 1) # Category


FAMILY : 18.97810218978102
GAME : 9.70241437394722
TOOLS : 8.433464345873105
BUSINESS : 4.581695676586187
LIFESTYLE : 3.9303761931499155
PRODUCTIVITY : 3.885457608085345
FINANCE : 3.6833239752947784
MEDICAL : 3.5148792813026386
SPORTS : 3.3801235261089273
PERSONALIZATION : 3.312745648512072
COMMUNICATION : 3.2341381246490735
HEALTH_AND_FITNESS : 3.065693430656934
PHOTOGRAPHY : 2.9421673217293653
NEWS_AND_MAGAZINES : 2.829870859067939
SOCIAL : 2.6501965188096577
TRAVEL_AND_LOCAL : 2.3245367770915215
SHOPPING : 2.2459292532285233
BOOKS_AND_REFERENCE : 2.1785513756316677
DATING : 1.8528916339135317
VIDEO_PLAYERS : 1.7967434025828188
MAPS_AND_NAVIGATION : 1.4149354295339696
FOOD_AND_DRINK : 1.235261089275688
EDUCATION : 1.167883211678832
ENTERTAINMENT : 0.9545199326221224
LIBRARIES_AND_DEMO : 0.9320606400898372
AUTO_AND_VEHICLES : 0.9208309938236946
HOUSE_AND_HOME : 0.8197641774284109
WEATHER : 0.7973048848961257
EVENTS : 0.7074677147669848
PARENTING : 0.6513194834362718
ART_AND_DESIGN : 0

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the Genres column:

display_table(android_final, -4)


The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

## Most Popular Apps by Genre on Google Play


In [75]:
display_table(android_final,5)

1,000,000+ : 15.687815833801237
100,000+ : 11.577765300393038
10,000,000+ : 10.499719258843346
10,000+ : 10.252667040988209
1,000+ : 8.422234699606962
100+ : 6.917462099943853
5,000,000+ : 6.816395283548568
500,000+ : 5.53621560920831
50,000+ : 4.817518248175182
5,000+ : 4.525547445255475
10+ : 3.537338573834924
500+ : 3.2341381246490735
50,000,000+ : 2.2908478382930935
100,000,000+ : 2.1224031443009546
50+ : 1.9090398652442448
5+ : 0.7860752386299831
1+ : 0.5165637282425604
500,000,000+ : 0.26951151038742277
1,000,000,000+ : 0.22459292532285235
0+ : 0.044918585064570464
0 : 0.011229646266142616


One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

To perform computations, however, we'll need to convert each install number to float — this means that we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error. We'll do this directly in the loop below, where we also compute the average number of installs for each genre (category).

In [77]:
categories_android = freq_table(android_final, 1)
categories_android

{'ART_AND_DESIGN': 0.6513194834362718,
 'AUTO_AND_VEHICLES': 0.9208309938236946,
 'BEAUTY': 0.5951712521055587,
 'BOOKS_AND_REFERENCE': 2.1785513756316677,
 'BUSINESS': 4.581695676586187,
 'COMICS': 0.6288601909039866,
 'COMMUNICATION': 3.2341381246490735,
 'DATING': 1.8528916339135317,
 'EDUCATION': 1.167883211678832,
 'ENTERTAINMENT': 0.9545199326221224,
 'EVENTS': 0.7074677147669848,
 'FINANCE': 3.6833239752947784,
 'FOOD_AND_DRINK': 1.235261089275688,
 'HEALTH_AND_FITNESS': 3.065693430656934,
 'HOUSE_AND_HOME': 0.8197641774284109,
 'LIBRARIES_AND_DEMO': 0.9320606400898372,
 'LIFESTYLE': 3.9303761931499155,
 'GAME': 9.70241437394722,
 'FAMILY': 18.97810218978102,
 'MEDICAL': 3.5148792813026386,
 'SOCIAL': 2.6501965188096577,
 'SHOPPING': 2.2459292532285233,
 'PHOTOGRAPHY': 2.9421673217293653,
 'SPORTS': 3.3801235261089273,
 'TRAVEL_AND_LOCAL': 2.3245367770915215,
 'TOOLS': 8.433464345873105,
 'PERSONALIZATION': 3.312745648512072,
 'PRODUCTIVITY': 3.885457608085345,
 'PARENTING': 0.6

In [78]:
android

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10835,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10836,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10837,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10838,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [83]:
Total=0
Length=0
for category in categories_android:
    for row in android_final:
        if category==row[1]:
            Length+=1
            Sub_total=row[5]
            Sub_total=Sub_total.replace("+"," ")
            Sub_total=Sub_total.replace(",","")
            Total+=float(Sub_total)
    Avg=Total/Length
    print (category,":",Avg)

ART_AND_DESIGN : 1952105.1724137932
AUTO_AND_VEHICLES : 1187872.5785714285
BEAUTY : 1002586.5854922279
BOOKS_AND_REFERENCE : 4804768.658914729
BUSINESS : 3215594.4163522013
COMICS : 3056849.249118684
COMMUNICATION : 11973920.02809482
DATING : 10566878.580521472
EDUCATION : 9921207.151278408
ENTERTAINMENT : 10019102.256530475
EVENTS : 9623710.044344474
FINANCE : 8189838.620488322
FOOD_AND_DRINK : 7844230.046138415
HEALTH_AND_FITNESS : 7404033.133656816
HOUSE_AND_HOME : 7214592.126068376
LIBRARIES_AND_DEMO : 6989327.851836566
LIFESTYLE : 6288418.97367472
GAME : 8489059.737146
FAMILY : 6959846.434203116
MEDICAL : 6580289.769326241
SOCIAL : 7249948.2983321985
SHOPPING : 7241776.64351547
PHOTOGRAPHY : 7677075.386715052
SPORTS : 7493980.190390119
TRAVEL_AND_LOCAL : 7690218.897166229
TOOLS : 7996351.591944189
PERSONALIZATION : 7891221.367460719
PRODUCTIVITY : 8262830.583394027
PARENTING : 8208855.9975892
WEATHER : 8182258.620294012
VIDEO_PLAYERS : 8489831.077283923
NEWS_AND_MAGAZINES : 851600

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [91]:
for app in android_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times:



In [93]:
under_100_m = []

for app in android_final:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3589717.245210728

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [95]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [97]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [ ]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

## Conclusions

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.